In [ ]:
#libraries
import requests
from io import BytesIO
import time
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import json
import pickle
from minio import Minio

In [ ]:
def get_latest_file_with_extension(extension):
    objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
    files = [obj for obj in objects if obj.object_name.endswith(extension)]
    if not files:
        return None
    files.sort(key=lambda x: x.last_modified, reverse=True)
    return files[0]
def show_image(obj_key,output):
    response = client.get_object(bucket_name, obj_key)
    image_data = response.read()

    # Save image
    os.makedirs(output, exist_ok=True)
    local_path = os.path.join("output", os.path.basename(obj_key))
    with open(local_path, "wb") as f:
        f.write(image_data)
    print(f"Image saved in: {local_path}")

    # Show image
    image = Image.open(BytesIO(image_data))
    plt.figure()
    plt.imshow(image)
    plt.axis('off')
    plt.title(os.path.basename(obj_key))
    plt.show()
def show_json(obj_key,output):
    response = client.get_object(bucket_name, obj_key)
    json_bytes = response.read()
    #Save json
    os.makedirs(output, exist_ok=True)
    local_path = os.path.join("output", os.path.basename(obj_key))
    with open(local_path, "wb") as f:
        f.write(json_bytes)
    print(f"JSON file saved in: {local_path}")
    # Show json file
    json_data = json.loads(json_bytes.decode("utf-8"))
    print(f"Content of {obj_key}:\n")
    print(json.dumps(json_data, indent=4))

In [ ]:
minio_credencials= os.getenv("MINIO_CREDENCIALS","minio_fish.pkl")

if not minio_credencials.endswith(".pkl"):
    minio_credencials += ".pkl"
# Load configuration from .pkl file
with open(minio_credencials, "rb") as f:
    config = pickle.load(f)
# Create MinIO client
client = Minio(
    endpoint=config["endpoint"],
    access_key=config["access_key"],
    secret_key=config["secret_key"],
    secure= True
)
print("MinIO client loaded from configuration.")

In [ ]:
#Necessary environment variables

bucket_name= os.getenv("BUCKET_NAME","")
out= os.getenv("PREFIX","")
prefix=out+"/"
output= os.getenv("OUTPUT","output")
download_dir = "./temporal"  # Local folder where to save the file


In [ ]:

print(f"Waiting for news .zip  and .jpg file in {bucket_name}/{prefix}...")

last_seen_json = get_latest_file_with_extension('.json')
last_seen_jpg = get_latest_file_with_extension('.jpg')
last_json_time = last_seen_json.last_modified if last_seen_json else None
last_jpg_time = last_seen_jpg.last_modified if last_seen_jpg else None
while True:
    current_json = get_latest_file_with_extension('.json')
    current_jpg = get_latest_file_with_extension('.jpg')
    
    new_json = (not last_seen_json or current_json.last_modified > last_json_time)
    new_jpg = (not last_seen_jpg or current_jpg.last_modified > last_jpg_time)

    if current_json and current_jpg and new_json and new_jpg:
        print(f"🆕 Detected file: {current_json['Key']} y {current_jpg['Key']}")
        # Download and show image
        show_json(current_jpg['Key'],output)
        # Download and show JSON
        show_jsons3(current_json['Key'],output)
        break
    time.sleep(5)